In [ ]:
from google.colab import drive


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/UNSW_NB15_training-set.csv')

In [ ]:
df.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, SimpleRNN
from keras.utils import to_categorical
from keras.layers import Dense

In [ ]:
label_encoder = LabelEncoder()
df['attack_cat_encoded'] = label_encoder.fit_transform(df['attack_cat'])
df = pd.get_dummies(df, columns=['proto', 'service', 'state'], drop_first=True)
df = df.drop(columns=['attack_cat', 'id'])

X = df.drop(columns=['label'])
y = df['attack_cat_encoded']
y_onehot = to_categorical(y)

In [ ]:
model = xgb.XGBClassifier(use_label_encoder=False)
model.fit(X, y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
feature_importances = model.feature_importances_
features_ranked = sorted(list(zip(X.columns, feature_importances)), key=lambda x: x[1], reverse=True)

print(features_ranked[:20])

[('service_dns', 0.7140337), ('attack_cat_encoded', 0.20601222), ('smean', 0.046286944), ('sttl', 0.032070447), ('dur', 0.0011605837), ('state_INT', 0.00016537105), ('rate', 7.174198e-05), ('sbytes', 6.4222e-05), ('ct_dst_sport_ltm', 3.3021686e-05), ('sloss', 1.5997563e-05), ('tcprtt', 1.3907831e-05), ('ct_srv_src', 1.0755306e-05), ('ct_src_dport_ltm', 1.016062e-05), ('sjit', 9.450075e-06), ('ct_dst_src_ltm', 6.980171e-06), ('synack', 5.404572e-06), ('dbytes', 5.3926456e-06), ('sload', 5.255364e-06), ('ct_srv_dst', 3.4756263e-06), ('dmean', 3.3087645e-06)]


In [ ]:
selected_features = [feature[0] for feature in features_ranked[:20]]
X_selected = X[selected_features]

In [ ]:
scaler = StandardScaler()
X_selected_scaled = scaler.fit_transform(X_selected)
X_selected_reshaped = X_selected_scaled.reshape(X_selected_scaled.shape[0], X_selected_scaled.shape[1], 1)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_selected_reshaped, y_onehot, test_size=0.2, random_state=42)

In [ ]:
from keras.layers import Input
from keras.models import Model

In [ ]:
encoding_dim = 50  # Define encoding dimension

# Encoder
input_layer = Input(shape=(X_train.shape[1],))
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# Decoder
decoded = Dense(X_train.shape[1], activation='sigmoid')(encoded)

# Autoencoder = Encoder + Decoder
autoencoder = Model(input_layer, decoded)

# Compile and train the autoencoder
autoencoder.compile(optimizer='adam', loss='mean_squared_error')
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_val, X_val))

# For classification: Remove the decoder and add classification layers to the encoder
encoder = Model(input_layer, encoded)
num_classes = y_onehot.shape[1]
classification_layer = Dense(num_classes, activation='softmax')(encoder.output)

# Final model for classification
final_model = Model(inputs=encoder.input, outputs=classification_layer)
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
final_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
2059/2059 [==============================] - 7s 3ms/step - loss: 0.6797 - val_loss: 0.6570
Epoch 2/10
2059/2059 [==============================] - 6s 3ms/step - loss: 0.6420 - val_loss: 0.6544
Epoch 3/10
2059/2059 [==============================] - 4s 2ms/step - loss: 0.6405 - val_loss: 0.6536
Epoch 4/10
2059/2059 [==============================] - 6s 3ms/step - loss: 0.6399 - val_loss: 0.6531
Epoch 5/10
2059/2059 [==============================] - 4s 2ms/step - loss: 0.6371 - val_loss: 0.6494
Epoch 6/10
2059/2059 [==============================] - 4s 2ms/step - loss: 0.6367 - val_loss: 0.6493
Epoch 7/10
2059/2059 [==============================] - 6s 3ms/step - loss: 0.6366 - val_loss: 0.6493
Epoch 8/10
2059/2059 [==============================] - 4s 2ms/step - loss: 0.6366 - val_loss: 0.6492
Epoch 9/10
2059/2059 [==============================] - 4s 2ms/step - loss: 0.6365 - val_loss: 0.6491
Epoch 10/10
2059/2059 [==============================] - 6s 3ms/step - loss: 0.636

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(50, input_shape=(X_train.shape[1], 1)))
lstm_model.add(Dense(y_onehot.shape[1], activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
2059/2059 [==============================] - 31s 14ms/step - loss: 0.3310 - accuracy: 0.9016 - val_loss: 0.0669 - val_accuracy: 0.9885
Epoch 2/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0686 - accuracy: 0.9831 - val_loss: 0.0364 - val_accuracy: 0.9926
Epoch 3/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0382 - accuracy: 0.9911 - val_loss: 0.0185 - val_accuracy: 0.9974
Epoch 4/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0204 - accuracy: 0.9964 - val_loss: 0.0096 - val_accuracy: 0.9985
Epoch 5/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0399 - accuracy: 0.9911 - val_loss: 0.0185 - val_accuracy: 0.9985
Epoch 6/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0219 - accuracy: 0.9962 - val_loss: 0.0341 - val_accuracy: 0.9893
Epoch 7/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0123 - accuracy: 0.9973 - val_loss: 0.0072 -

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(50, input_shape=(X_train.shape[1], 1)))
gru_model.add(Dense(y_onehot.shape[1], activation='softmax'))
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
2059/2059 [==============================] - 32s 15ms/step - loss: 0.4031 - accuracy: 0.8812 - val_loss: 0.0558 - val_accuracy: 0.9916
Epoch 2/10
2059/2059 [==============================] - 28s 14ms/step - loss: 0.0398 - accuracy: 0.9931 - val_loss: 0.0187 - val_accuracy: 0.9973
Epoch 3/10
2059/2059 [==============================] - 28s 14ms/step - loss: 0.0505 - accuracy: 0.9893 - val_loss: 0.1717 - val_accuracy: 0.9227
Epoch 4/10
2059/2059 [==============================] - 30s 14ms/step - loss: 0.0200 - accuracy: 0.9956 - val_loss: 0.0095 - val_accuracy: 0.9983
Epoch 5/10
2059/2059 [==============================] - 28s 14ms/step - loss: 0.0067 - accuracy: 0.9985 - val_loss: 0.0065 - val_accuracy: 0.9988
Epoch 6/10
2059/2059 [==============================] - 27s 13ms/step - loss: 0.0153 - accuracy: 0.9968 - val_loss: 0.0040 - val_accuracy: 0.9991
Epoch 7/10
2059/2059 [==============================] - 28s 14ms/step - loss: 0.0136 - accuracy: 0.9970 - val_loss: 0.0087 -

In [ ]:
rnn_model = Sequential()
rnn_model.add(SimpleRNN(50, input_shape=(X_train.shape[1], 1)))
rnn_model.add(Dense(y_onehot.shape[1], activation='softmax'))
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/10
2059/2059 [==============================] - 14s 7ms/step - loss: 0.1321 - accuracy: 0.9641 - val_loss: 0.0212 - val_accuracy: 0.9931
Epoch 2/10
2059/2059 [==============================] - 14s 7ms/step - loss: 0.0162 - accuracy: 0.9958 - val_loss: 0.0072 - val_accuracy: 0.9987
Epoch 3/10
2059/2059 [==============================] - 14s 7ms/step - loss: 0.0095 - accuracy: 0.9976 - val_loss: 0.0063 - val_accuracy: 0.9991
Epoch 4/10
2059/2059 [==============================] - 15s 7ms/step - loss: 0.0112 - accuracy: 0.9973 - val_loss: 0.0108 - val_accuracy: 0.9979
Epoch 5/10
2059/2059 [==============================] - 15s 8ms/step - loss: 0.0087 - accuracy: 0.9980 - val_loss: 0.0070 - val_accuracy: 0.9987
Epoch 6/10
2059/2059 [==============================] - 14s 7ms/step - loss: 0.0097 - accuracy: 0.9977 - val_loss: 0.0034 - val_accuracy: 0.9994
Epoch 7/10
2059/2059 [==============================] - 15s 7ms/step - loss: 0.0087 - accuracy: 0.9980 - val_loss: 0.0036 - val_ac

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [ ]:
def evaluate_model(model, X, y_true, model_name):
    # Get model predictions
    y_pred_probs = model.predict(X)
    y_pred = y_pred_probs.argmax(axis=1)
    y_true_labels = y_true.argmax(axis=1)

    # Calculate metrics
    accuracy = accuracy_score(y_true_labels, y_pred)
    precision = precision_score(y_true_labels, y_pred, average='macro')
    recall = recall_score(y_true_labels, y_pred, average='macro')
    f1 = f1_score(y_true_labels, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_true_labels, y_pred)

    # Display metrics
    print(f"Performance Metrics for {model_name}:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("="*60)

In [ ]:
models = [final_model, lstm_model, gru_model, rnn_model]  # Replace with your model instances
model_names = ["Autoencoder-based Classifier", "LSTM", "GRU", "RNN"]

for model, name in zip(models, model_names):
    evaluate_model(model, X_val, y_val, name)

515/515 [==============================] - 1s 1ms/step
Performance Metrics for Autoencoder-based Classifier:
Accuracy: 0.9996
Precision: 0.9983
Recall: 0.9996
F1-Score: 0.9989
Confusion Matrix:
[[ 131    0    0    0    0    0    0    0    0    0]
 [   0  117    0    0    0    0    0    0    0    0]
 [   0    0  786    0    0    0    0    0    0    0]
 [   0    0    3 2272    0    0    0    0    0    0]
 [   0    0    0    0 1212    0    0    0    0    0]
 [   0    0    0    0    0 3722    1    0    0    0]
 [   0    0    0    0    0    0 7418    0    0    0]
 [   0    0    0    0    0    0    1  721    1    0]
 [   0    0    0    0    0    0    0    0   75    0]
 [   0    0    0    0    0    0    0    0    0    7]]
515/515 [==============================] - 2s 4ms/step
Performance Metrics for LSTM:
Accuracy: 0.9996
Precision: 0.9998
Recall: 0.9993
F1-Score: 0.9996
Confusion Matrix:
[[ 131    0    0    0    0    0    0    0    0    0]
 [   0  117    0    0    0    0    0    0    0    0]

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/UNSW_NB15_testing-set.csv')

In [ ]:
df_test['attack_cat_encoded'] = label_encoder.transform(df_test['attack_cat'])
df_test = pd.get_dummies(df_test, columns=['proto', 'service', 'state'], drop_first=True)
df_test = df_test.drop(columns=['attack_cat', 'id'])

In [ ]:
X_test_data = df_test.drop(columns=['label'])
y_test_data = df_test['attack_cat_encoded']
y_test_data_onehot = to_categorical(y_test_data)

In [ ]:
selected_features = [feature[0] for feature in features_ranked[:20]]

In [ ]:
X_test_data_selected = X_test_data[selected_features]

In [ ]:
X_test_data_selected_scaled = scaler.transform(X_test_data_selected)
X_test_data_selected_reshaped = X_test_data_selected_scaled.reshape(X_test_data_selected_scaled.shape[0], X_test_data_selected_scaled.shape[1], 1)

In [ ]:
evaluation_results_selected = []

In [ ]:
for model, name in zip(models, model_names):
    # Get model predictions
    y_pred_probs = model.predict(X_test_data_selected_reshaped)
    y_pred = y_pred_probs.argmax(axis=1)
    y_true_labels = y_test_data_onehot.argmax(axis=1)

    # Calculate metrics
    accuracy = accuracy_score(y_true_labels, y_pred)
    precision = precision_score(y_true_labels, y_pred, average='macro')
    recall = recall_score(y_true_labels, y_pred, average='macro')
    f1 = f1_score(y_true_labels, y_pred, average='macro')

    evaluation_results_selected.append({
        "Model": name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1
    })

evaluation_df_selected = pd.DataFrame(evaluation_results_selected)

evaluation_df_selected

5480/5480 [==============================] - 15s 3ms/step


,Model,Accuracy,Precision,Recall,F1-Score
0,Autoencoder-based Classifier,0.997508,0.987664,0.986540,0.987052
1,LSTM,0.997645,0.991203,0.994551,0.992817
2,GRU,0.999013,0.997413,0.995542,0.996445
3,RNN,0.988029,0.942408,0.889135,0.867235
